# Eaton & Ree (2013) single-end RAD data set
Here we demonstrate a denovo assembly for an empirical RAD data set using the ipyrad Python API. This example was run on a workstation with 20 cores available and takes about 5 minutes to run completely, but can be run on even a laptop in about half an hour.

We will use the 13 taxa *Pedicularis* data set from [Eaton and Ree (2013) (Open Access)](https://academic.oup.com/sysbio/article/62/5/689/1684460/Inferring-Phylogeny-and-Introgression-using-RADseq). This data set is composed of single-end 75bp reads from a RAD-seq library prepared with the PstI enzyme. The data set also serves as an example for several of our analysis cookbooks that demonstrate methods for analyzing RAD-seq results. At the end of this notebook we also demonstrate the use of ipyrad.analysis tools to run downstream analyses. 

<img  src="https://github.com/eaton-lab/eaton-lab.github.io/blob/master/slides/slide-images/Eaton-Ree-2012-Ped-Fig1.png">

## Setup (software and data files)
If you haven't done so yet, start by installing `ipyrad` using conda (see ipyrad installation instructions). To use the ipyrad API you should then start a `jupyter-notebook`, like this one, and an `ipcluster` instance in a separate terminal, which we will use to parallelize our code. Then start your notebook by providing some text (like above) and by importing the packages below, including ipyrad. 

In [1]:
## conda install ipyrad -c ipyrad
## conda install toytree -c eaton-lab
## conda install sratools -c bioconda
## conda install entrez-direct -c bioconda

In [2]:
## ipcluster start --n=20

In [1]:
## imports
import ipyrad as ip
import ipyrad.analysis as ipa
import ipyparallel as ipp

In contrast to the ipyrad CLI, the ipyrad API requires requires users to start the parallel client themselves, which allows for more fine-scale control of the parallelization. This is done by starting a *parallel client* by running the command `ipcluster start` in a terminal. This will start a number of engines (independent ipython kernels) which parallel work will be farmed out to. It sounds a little complicated, but it's a very powerful way to parallelize code, and worth learning a little bit about. We aim to make it quite easy to use with ipyrad.  

In [2]:
## connect to cluster
ipyclient = ipp.Client()
print ipyclient.ids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


### Download the data set (Pedicularis)
These data are archived on the NCBI sequence read archive (SRA) under accession id SRP021469. For convenience, the data are also hosted at a public Dropbox link which is a bit easier to access. Run the code below to download and decompress the fastq data files, which will save them into a directory called example_empirical_data/. The compressed file size is approximately 1.1GB.

In [5]:
## download the Pedicularis data set from NCBI
sra = ipa.sratools(accession="SRP021469", workdir="fastqs-Ped")
sra.run(force=True, ipyclient=ipyclient)

[####################] 100%  Downloading fastq files | 0:01:19 |  
13 fastq files downloaded to /home/deren/Documents/ipyrad/tests/fastqs-Ped


## Create an Assembly object
This object stores the parameters of the assembly and the organization of data files. 

In [6]:
## you must provide a name for the Assembly
data = ip.Assembly("pedicularis")

New Assembly: pedicularis


Set parameters for the Assembly. This will raise an error if any of the parameters are not allowed because they are the wrong type, or out of the allowed range. 

In [10]:
## set parameters
data.set_params("project_dir", "analysis-ipyrad")
data.set_params("sorted_fastq_path", "fastqs-Ped/*.fastq.gz")
data.set_params("clust_threshold", "0.90")
data.set_params("filter_adapters", "2")
data.set_params("max_Hs_consens", (5, 5))
data.set_params("trim_loci", (0, 5, 0, 0))
data.set_params("output_formats", "psvnku")

## see/print all parameters
data.get_params()

0   assembly_name               pedicularis                                  
1   project_dir                 ./analysis-ipyrad                            
2   raw_fastq_path                                                           
3   barcodes_path                                                            
4   sorted_fastq_path           ./example_empirical_rad/*.fastq.gz           
5   assembly_method             denovo                                       
6   reference_sequence                                                       
7   datatype                    rad                                          
8   restriction_overhang        ('TGCAG', '')                                
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         33                                           
11  mindepth_statistical        6                                            
12  mindepth_majrule            6                               

## Assemble the data set


In [24]:
## run steps 1 & 2 of the assembly
data.run("12")

Assembly: pedicularis
[####################] 100%  loading reads         | 0:00:04 | s1 | 
[####################] 100%  processing reads      | 0:01:14 | s2 | 


In [25]:
## access the stats of the assembly (so far) from the .stats attribute
data.stats

,state,reads_raw,reads_passed_filter
29154_superba,2,696994,689996
30556_thamno,2,1452316,1440314
30686_cyathophylla,2,1253109,1206947
32082_przewalskii,2,964244,955480
33413_thamno,2,636625,626084
33588_przewalskii,2,1002923,993873
35236_rex,2,1803858,1787366
35855_rex,2,1409843,1397068
38362_rex,2,1391175,1379626
39618_rex,2,822263,813990


In [26]:
## run steps 3-6 of the assembly
data.run("3456")

Assembly: pedicularis
[####################] 100%  dereplicating         | 0:00:07 | s3 | 
[####################] 100%  clustering            | 0:05:08 | s3 | 
[####################] 100%  building clusters     | 0:00:26 | s3 | 
[####################] 100%  chunking              | 0:00:05 | s3 | 
[####################] 100%  aligning              | 0:03:06 | s3 | 
[####################] 100%  concatenating         | 0:00:15 | s3 | 
[####################] 100%  inferring [H, E]      | 0:01:10 | s4 | 
[####################] 100%  calculating depths    | 0:00:06 | s5 | 
[####################] 100%  chunking clusters     | 0:00:06 | s5 | 
[####################] 100%  consens calling       | 0:01:56 | s5 | 
[####################] 100%  concat/shuffle input  | 0:00:05 | s6 | 
[####################] 100%  clustering across     | 0:03:47 | s6 | 
[####################] 100%  building clusters     | 0:00:06 | s6 | 
[####################] 100%  aligning clusters     | 0:00:29 | s6 | 
[###########

### Branch to create several final data sets with different parameter settings



In [11]:
## create a branch for outputs with min_samples = 4 (lots of missing data)
min4 = data.branch("min4")
min4.set_params("min_samples_locus", 4)
min4.run("7")

## create a branch for outputs with min_samples = 13 (no missing data)
min13 = data.branch("min13")
min13.set_params("min_samples_locus", 13)
min13.run("7")

## create a branch with no missing data for ingroups, but allow
## missing data in the outgroups by setting population assignments.
## The population min-sample values overrule the min-samples-locus param
pops = data.branch("min11-pops")
pops.populations = {
    "ingroup": (11, [i for i in pops.samples if "prz" not in i]),
    "outgroup" : (0, [i for i in pops.samples if "prz" in i]),
    }
pops.run("7")

## create a branch with no missing data and with outgroups removed
nouts = data.branch("nouts_min11", subsamples=[i for i in pops.samples if "prz" not in i])
nouts.set_params("min_samples_locus", 11)
nouts.run("7")

Assembly: min4
[####################] 100%  filtering loci        | 0:00:08 | s7 | 
[####################] 100%  building loci/stats   | 0:00:01 | s7 | 
[####################] 100%  building vcf file     | 0:00:09 | s7 | 
[####################] 100%  writing vcf file      | 0:00:00 | s7 | 
[####################] 100%  building arrays       | 0:00:04 | s7 | 
[####################] 100%  writing outfiles      | 0:00:05 | s7 | 
Outfiles written to: ~/Documents/ipyrad/tests/analysis-ipyrad/min4_outfiles

Assembly: min13
[####################] 100%  filtering loci        | 0:00:02 | s7 | 
[####################] 100%  building loci/stats   | 0:00:01 | s7 | 
[####################] 100%  building vcf file     | 0:00:03 | s7 | 
[####################] 100%  writing vcf file      | 0:00:00 | s7 | 
[####################] 100%  building arrays       | 0:00:04 | s7 | 
[####################] 100%  writing outfiles      | 0:00:00 | s7 | 
Outfiles written to: ~/Documents/ipyrad/tests/analysis-ipyrad/mi

### View final stats
The .stats attribute shows a stats summary for each sample, and a number of stats dataframes can be accessed for each step from the .stats_dfs attribute of the Assembly. 

In [9]:
## we can access the stats summary as a pandas dataframes. 
min4.stats

,state,reads_raw,reads_passed_filter,clusters_total,clusters_hidepth,hetero_est,error_est,reads_consens
29154_superba,6,696994,689996,134897,35143,0.010,0.002,34628
30556_thamno,6,1452316,1440314,212960,52776,0.011,0.003,51701
30686_cyathophylla,6,1253109,1206947,240824,54282,0.010,0.002,53347
32082_przewalskii,6,964244,955480,151762,42487,0.013,0.002,41841
33413_thamno,6,636625,626084,172709,31072,0.012,0.002,30674
33588_przewalskii,6,1002923,993873,158933,46555,0.013,0.002,45892
35236_rex,6,1803858,1787366,417632,54709,0.010,0.001,54064
35855_rex,6,1409843,1397068,184661,56595,0.013,0.003,55371
38362_rex,6,1391175,1379626,133541,53156,0.008,0.002,52496
39618_rex,6,822263,813990,148107,44016,0.010,0.002,43409


In [25]:
## or print the full stats file 
cat $min4.stats_files.s7



## The number of loci caught by each filter.
## ipyrad API location: [assembly].stats_dfs.s7_filters

                            total_filters  applied_order  retained_loci
total_prefiltered_loci              96562              0          96562
filtered_by_rm_duplicates            2939           2939          93623
filtered_by_max_indels                189            189          93434
filtered_by_max_snps                  679             18          93416
filtered_by_max_shared_het            865            708          92708
filtered_by_min_sample              47958          46646          46062
filtered_by_max_alleles             10694           4565          41497
total_filtered_loci                 41497              0          41497


## The number of loci recovered for each Sample.
## ipyrad API location: [assembly].stats_dfs.s7_samples

                        sample_coverage
29154_superba                     21054
30556_thamno                      31790
30686_cyathophylla  

In [13]:
## and we can access parts of the full stats outputs as dataframes 
min4.stats_dfs.s7_samples

,sample_coverage
29154_superba,21054
30556_thamno,31790
30686_cyathophylla,26648
32082_przewalskii,13201
33413_thamno,18747
33588_przewalskii,15461
35236_rex,33228
35855_rex,33403
38362_rex,33748
39618_rex,28135


In [14]:
## compare this to the one above, coverage is more equal
min13.stats_dfs.s7_samples

,sample_coverage
29154_superba,2833
30556_thamno,2833
30686_cyathophylla,2833
32082_przewalskii,2833
33413_thamno,2833
33588_przewalskii,2833
35236_rex,2833
35855_rex,2833
38362_rex,2833
39618_rex,2833


In [15]:
## similarly, coverage is equal here among ingroups, but allows missing in outgroups
pops.stats_dfs.s7_samples

,sample_coverage
29154_superba,5796
30556_thamno,5796
30686_cyathophylla,5796
32082_przewalskii,3266
33413_thamno,5796
33588_przewalskii,3747
35236_rex,5796
35855_rex,5796
38362_rex,5796
39618_rex,5796


## Analysis tools
We have a lot more information about analysis tools in the ipyrad documentation. But here I'll show just a quick example of how you can easily access the data files for these assemblies and use them in downstream analysis software. The ipyrad analysis tools include convenient wrappers to make it easier to parallelize analyses of RAD-seq data. Please see the full documentation for the ipyrad.analysis tools in the ipyrad documentation for more details. 

In [3]:
import ipyrad as ip
import ipyrad.analysis as ipa

In [4]:
## you can re-load assemblies at a later time from their JSON file
min4 = ip.load_json("analysis-ipyrad/min4.json")
min13 = ip.load_json("analysis-ipyrad/min13.json")
nouts = ip.load_json("analysis-ipyrad/nouts_min11.json")

loading Assembly: min4
from saved path: ~/Documents/ipyrad/tests/analysis-ipyrad/min4.json
loading Assembly: min13
from saved path: ~/Documents/ipyrad/tests/analysis-ipyrad/min13.json
loading Assembly: nouts_min11
from saved path: ~/Documents/ipyrad/tests/analysis-ipyrad/nouts_min11.json


#### *RAxML*  -- ML concatenation tree inference

In [17]:
## conda install raxml -c bioconda
## conda install toytree -c eaton-lab

In [12]:
## create a raxml analysis object for the min13 data sets
rax = ipa.raxml(
    name=min13.name, 
    data=min13.outfiles.phy,
    workdir="analysis-raxml",
    T=20,
    N=100,    
    o=[i for i in min13.samples if "prz" in i],
    )

In [19]:
## print the raxml command and call it
print rax.command
rax.run(force=True)

raxmlHPC-PTHREADS-SSE3 -f a -T 20 -m GTRGAMMA -N 100 -x 12345 -p 54321 -n min13 -w /home/deren/Documents/ipyrad/tests/analysis-raxml -s /home/deren/Documents/ipyrad/tests/analysis-ipyrad/min13_outfiles/min13.phy -o 33588_przewalskii,32082_przewalskii
job min13 finished successfully


In [13]:
## access the resulting tree files
rax.trees

bestTree                   ~/Documents/ipyrad/tests/analysis-raxml/RAxML_bestTree.min13
bipartitions               ~/Documents/ipyrad/tests/analysis-raxml/RAxML_bipartitions.min13
bipartitionsBranchLabels   ~/Documents/ipyrad/tests/analysis-raxml/RAxML_bipartitionsBranchLabels.min13
bootstrap                  ~/Documents/ipyrad/tests/analysis-raxml/RAxML_bootstrap.min13
info                       ~/Documents/ipyrad/tests/analysis-raxml/RAxML_info.min13

In [15]:
## plot a tree in the notebook with toytree
import toytree
tre = toytree.tree(rax.trees.bipartitions)
tre.draw(
    width=350,
    height=400,
    node_labels=tre.get_node_values("support"),
    );

32082_przewalskii 33588_przewalskii 29154_superba 30686_cyathophylla 41478_cyathophylloides 41954_cyathophylloides 33413_thamno 30556_thamno 35855_rex 40578_rex 35236_rex 39618_rex 38362_rex idx: 1
name: 1
dist: 0.0112591855114
support: 100 100 idx: 2
name: 2
dist: 0.0112591855114
support: 100 100 idx: 3
name: 3
dist: 0.00138635903129
support: 100 100 idx: 4
name: 4
dist: 0.00130362717349
support: 100 100 idx: 5
name: 5
dist: 0.00515136675257
support: 100 100 idx: 6
name: 6
dist: 0.0026758642527
support: 100 100 idx: 7
name: 7
dist: 0.000567990916626
support: 77 77 idx: 8
name: 8
dist: 0.000342308540455
support: 73 73 idx: 9
name: 9
dist: 0.00100552991932
support: 100 100 idx: 10
name: 10
dist: 0.000365735718032
support: 48 48 idx: 11
name: 11
dist: 0.0030022742397
support: 100 100

### *tetrad* -- quartet tree inference 

In [16]:
## create a tetrad analysis object
tet = ipa.tetrad(
    name=min4.name, 
    seqfile=min4.outfiles.snpsphy,
    mapfile=min4.outfiles.snpsmap,
    nboots=100,
    )

loading seq array [13 taxa x 174180 bp]
max unlinked SNPs per quartet (nloci): 39149


In [18]:
## run tree inference
tet.run(ipyclient)

host compute node: [20 cores] on tinus
inferring 715 induced quartet trees
[####################] 100%  initial tree | 0:00:05 |  
[####################] 100%  boot 100     | 0:02:23 |  


In [19]:
## access tree files
tet.trees

boots   ~/Documents/ipyrad/tests/analysis-tetrad/min4.boots
cons    ~/Documents/ipyrad/tests/analysis-tetrad/min4.cons
nhx     ~/Documents/ipyrad/tests/analysis-tetrad/min4.nhx
tree    ~/Documents/ipyrad/tests/analysis-tetrad/min4.tree

In [22]:
## plot results (just like above, but unrooted by default)
import toytree
tre = toytree.tree(tet.trees.nhx)
tre.root(wildcard="prz")
tre.draw(
    width=350,
    height=400,
    node_labels=tre.get_node_values("support"),
    );

33588_przewalskii 32082_przewalskii 30686_cyathophylla 29154_superba 41954_cyathophylloides 41478_cyathophylloides 33413_thamno 35236_rex 30556_thamno 35855_rex 40578_rex 38362_rex 39618_rex idx: 1
name: 1
dist: 100
support: 100 100 idx: 2
name: added-node
dist: 100
support: 100 100 idx: 3
name: 2
dist: 100
support: 100 100 idx: 4
name: 3
dist: 100
support: 100 100 idx: 5
name: 4
dist: 100
support: 100 100 idx: 6
name: 5
dist: 100
support: 100 100 idx: 7
name: 6
dist: 99
support: 99 99 idx: 8
name: 7
dist: 98
support: 98 98 idx: 9
name: 8
dist: 85
support: 85 85 idx: 10
name: 9
dist: 100
support: 100 100 idx: 11
name: 10
dist: 100
support: 100 100

#### *STRUCTURE* -- population cluster inference

In [ ]:
## conda install structure clumpp -c ipyrad

In [25]:
## create a structure analysis object for the no-outgroup data set
struct = ipa.structure(
    name=nouts.name, 
    data=nouts.outfiles.str, 
    mapfile=nouts.outfiles.snpsmap,    
)

## set params for analysis (should be longer in real analyses)
struct.mainparams.burnin=1000
struct.mainparams.numreps=8000

In [96]:
## run structure across 10 random replicates of sampled unlinked SNPs
for kpop in [2, 3, 4, 5, 6]:
    struct.run(kpop=kpop, nreps=10, ipyclient=ipyclient)

submitted 10 structure jobs [nouts_min11-K-2]
submitted 10 structure jobs [nouts_min11-K-3]
submitted 10 structure jobs [nouts_min11-K-4]
submitted 10 structure jobs [nouts_min11-K-5]
submitted 10 structure jobs [nouts_min11-K-6]


In [115]:
## wait for all of these jobs to finish
ipyclient.wait()

True

In [26]:
## collect results
tables = {}
for kpop in [2, 3, 4, 5, 6]:
    tables[kpop] = struct.get_clumpp_table(kpop)

mean scores across 10 replicates.
mean scores across 10 replicates.
mean scores across 10 replicates.
mean scores across 10 replicates.
mean scores across 10 replicates.


In [27]:
## custom sorting order
myorder = [
    "41478_cyathophylloides", 
    "41954_cyathophylloides", 
    "29154_superba",
    "30686_cyathophylla", 
    "33413_thamno", 
    "30556_thamno", 
    "35236_rex", 
    "40578_rex", 
    "35855_rex",
    "39618_rex", 
    "38362_rex",
]

In [33]:
## import toyplot (packaged with toytree) 
import toyplot

## plot bars for each K-value (mean of 10 reps)
for kpop in [2, 3, 4, 5, 6]:
    table = tables[kpop]
    table = table.ix[myorder]
    
    ## plot barplot w/ hover
    canvas, axes, mark = toyplot.bars(
                            table, 
                            title=[[i] for i in table.index.tolist()],
                            width=400, 
                            height=200, 
                            yshow=False,                            
                            style={"stroke": toyplot.color.near_black},
                            )

41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 0 5 10

41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 0 5 10

41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 0 5 10

41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 0 5 10

41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 41478_cyathophylloides 41954_cyathophylloides 29154_superba 30686_cyathophylla 33413_thamno 30556_thamno 35236_rex 40578_rex 35855_rex 39618_rex 38362_rex 0 5 10

#### *TREEMIX* -- ML tree & admixture co-inference

In [15]:
## conda install treemix -c ipyrad

In [37]:
## group taxa into 'populations'
imap = {
    "prz": ["32082_przewalskii", "33588_przewalskii"],
    "cys": ["41478_cyathophylloides", "41954_cyathophylloides"],
    "cya": ["30686_cyathophylla"],
    "sup": ["29154_superba"],
    "cup": ["33413_thamno"],
    "tha": ["30556_thamno"],
    "rck": ["35236_rex"],
    "rex": ["35855_rex", "40578_rex"],
    "lip": ["39618_rex", "38362_rex"],  
}

## optional: loci will be filtered if they do not have data for at
## least N samples in each species. Minimums cannot be <1.
minmap = {
    "prz": 2,
    "cys": 2,
    "cya": 1,
    "sup": 1,
    "cup": 1,
    "tha": 1, 
    "rck": 1,
    "rex": 2,
    "lip": 2,
    }

## create a treemix analysis object
tmix = ipa.treemix(
    name=min13.name, 
    data=min13.outfiles.snpsphy,
    mapfile=min13.outfiles.snpsmap,
    imap=imap,
    minmap=minmap,  
    )

## you can set additional parameter args here
tmix.params.m = 1
tmix.params.root = "prz"
tmix.params.global_ = 1
tmix.params

binary      treemix             
bootstrap   0                   
climb       0                   
cormig      0                   
g           (None, None)        
global_     1                   
k           0                   
m           1                   
noss        0                   
root        prz                 
se          0                   
seed        747538471           

In [38]:
## a dictionary for storing treemix objects
tdict = {}

## iterate over values of m
for rep in xrange(4):
    for mig in xrange(4):
        
        ## create new treemix object copy
        name = "mig-{}-rep-{}".format(mig, rep)
        tmp = tmix.copy(name)

        ## set params on new object
        tmp.params.m = mig
    
        ## run treemix analysis
        tmp.run()
        
        ## store the treemix object
        tdict[name] = tmp

In [39]:
## select a result
tmp = tdict["mig-2-rep-1"]

## draw the tree similar to the Treemix plotting R code
canvas = toyplot.Canvas(width=350, height=350)
axes = canvas.cartesian(padding=25, margin=75)
axes = tmp.draw(axes)#treemix_plot(tmp, axes)

NameError: name 'treemix_plot' is not defined

-0.5 0.0 0.5 -0.5 0.0 0.5

#### ABBA-BABA admixture inference

In [41]:
bb = ipa.baba(
    data=min4.outfiles.loci,
    newick=tre.tree.write(),   
)

In [50]:
## generate tests from the tree
bb.generate_tests_from_tree(
    constraint_dict={"p4": "32082_przewalskii"},
    constraint_exact=True,
    )

## show tests
bb.tests

0 tests generated from tree


[]

In [51]:
## run the tests in parallel
bb.run(ipyclient=ipyclient)

0[####################] 100%  calculating D-stats  | 0:00:00 |  


In [52]:
bb.results_table

,dstat,bootmean,bootstd,Z,ABBA,BABA,nloci
